# Exploratory Data Analysis

### Apa goals dari mempelajari EDA?

Goals dari EDA adalah mengenal data menggunakan beberapa teknik pengolahan data secara statistik maupun visual sehingga dapat memperoleh *insight* yang bermanfaat untuk menjawab pertanyaan bisnis.

## Datetime

### Apakah perbedaan antara `pd.to_datetime()` dengan `astype(datetime64)`?

Method `pd.to_datetime()` lebih fleksible jika dibandingkan dengan `astype(datetime64)`.
Hal ini dikarenakan method `pd.to_datetime()`memiliki lebih banyak parameter didalamnya. Contohnya saja, method ini dapat melakukan transformasi bentuk datetime dengan menambahkan parameter `format` dalam melakukan formatting urutan tanggal. Eg:`pd.to_datetime('30-11-2020', format='%d-%m-%Y')`.

### Bagaimana cara melihat selisih/jeda tanggal ?

Selisih tanggal dapat dihitung menggunakan *method* `timedelta` yang disediakan oleh *library* `datetime`.
Berikut dibawah ini adalah ilustrasi penggunaannya:

In [47]:
import pandas as pd

household = pd.read_csv('data_input/household.csv')
household['purchase_time'] = pd.to_datetime(household['purchase_time'])
household['weekday'] = household['purchase_time'].dt.day_name()

In [48]:
household.head(2)

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth,weekday
0,9622257,32369294,2018-07-22 21:19:00,Rice,Rice,supermarket,"128,000.00",0,1,2018-07,Sunday
1,9446359,31885876,2018-07-15 16:17:00,Rice,Rice,minimarket,"102,750.00",0,1,2018-07,Sunday


Tabel di atas adalah sebuah DataFrame `household` yang memuat informasi transaksi kebutuhan rumah tangga. Kolom `purchase_time` berisi informasi tanggal pembelian barang. Jika diketahui durasi pengiriman sebuah barang adalah 2 hari setelah barang dibeli, maka kita dapat menghitung estimasi barang sampai ke tangan pembeli dengan menggunakan *syntax* berikut:

In [49]:
from datetime import timedelta
household['shipdate_est'] = household['purchase_time'] + timedelta(days=2)
household.head(2)

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth,weekday,shipdate_est
0,9622257,32369294,2018-07-22 21:19:00,Rice,Rice,supermarket,"128,000.00",0,1,2018-07,Sunday,2018-07-24 21:19:00
1,9446359,31885876,2018-07-15 16:17:00,Rice,Rice,minimarket,"102,750.00",0,1,2018-07,Sunday,2018-07-17 16:17:00


### Apakah ekspresi dalam timedelta dapat dibuat lebih spesifik? Misalnya :(timedelta) 2 days 1 hours and 45 minutes?

Ya bisa. Berikut adalah contohnya :

In [58]:
household['shipdate_est_new'] = household['purchase_time'] + timedelta(days=2, hours=1, minutes=45)
household.head(2)

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth,weekday,shipdate_est,shipdate_est_new
0,9622257,32369294,2018-07-22 21:19:00,Rice,Rice,supermarket,"128,000.00",0,1,2018-07,Sunday,2018-07-24 21:19:00,2018-07-24 23:04:00
1,9446359,31885876,2018-07-15 16:17:00,Rice,Rice,minimarket,"102,750.00",0,1,2018-07,Sunday,2018-07-17 16:17:00,2018-07-17 18:02:00


### Ketika menggunakan perintah `dt.to_period('W')`, maka secara otomatis rentang *week* yang ditampilkan selalu dimulai dari hari Senin. Apakah ada cara untuk menampilkan rentang *week* yang dimulai dari hari Minggu

In [50]:
household['purchase_time'] = household['purchase_time'].astype('datetime64')

Apabila ingin menampilkan rentang *week* yang dimulai dari hari Minggu, maka dapat dituliskan syntax sebagai berikut :

In [56]:
household['purchase_time'].dt.to_period('W-SAT')

0        2018-07-22/2018-07-28
1        2018-07-15/2018-07-21
2        2018-07-15/2018-07-21
3        2018-07-22/2018-07-28
4        2018-07-22/2018-07-28
                 ...          
71995    2017-12-24/2017-12-30
71996    2017-12-10/2017-12-16
71997    2017-12-24/2017-12-30
71998    2017-12-03/2017-12-09
71999    2017-12-17/2017-12-23
Name: purchase_time, Length: 72000, dtype: period[W-SAT]

### Bagaimana cara mengubah nama hari atau bulan kedalam format Bahasa Indonesia?

Cara terbaik untuk mengubah nama hari/bulan kedalam Bahasa adalah dengan mengubah *locale time* terlebih dahulu, kemudian gunakan *method* `strftime()` untuk mengekstrak nama hari/bulannya

In [55]:
import locale
locale.setlocale(locale.LC_TIME, "id")

household['purchase_time'].dt.strftime("%A")

0        Minggu
1        Minggu
2        Minggu
3        Selasa
4         Kamis
          ...  
71995      Rabu
71996      Rabu
71997      Rabu
71998     Kamis
71999    Selasa
Name: purchase_time, Length: 72000, dtype: object

## Category

### Bagaimana cara mengetahui level pada category?

Cara mengetahui level pada category dapat menggunakan attribut `cat.categories` atau dapat menggunakan *method* `unique()`.
Berikut adalah contoh implementasinya :

In [60]:
household[['category','sub_category','format','yearmonth','weekday']] = \
household[['category','sub_category','format','yearmonth','weekday']].\
astype('category')

In [61]:
household['format'].cat.categories

Index(['hypermarket', 'minimarket', 'supermarket'], dtype='object')

In [62]:
household['format'].unique()

['supermarket', 'minimarket', 'hypermarket']
Categories (3, object): ['supermarket', 'minimarket', 'hypermarket']

**Apa yang membedakan `cat.categories` dengan `unique()`**

`cat.categories` adalah atribut milik tipe data categorical, sehingga tidak dapat digunakan di luar tipe data categorical. Sedangkan `unique()` adalah *method* yang dimiliki oleh `pandas` sehingga dapat digunakan dengan lebih fleksible sesuai dengan kebutuhan.

## Contingency Table

### Apakah parameter `aggfunc` pada crosstab dapat menampilkan lebih dari satu fungsi aggregasi?

Bisa. Parameter `aggfunc` dapat menampilkan lebih dari satu fungsi aggregasi. Berikut di bawah ini adalah contoh implementasi menampilkan tabulasi silang pada data menggunakan aggregasi `mean` dan `sum` :

In [63]:
pd.crosstab(
    index=household['category'],
    columns=household['format'],
    values=household['unit_price'],
    aggfunc='mean'
)

format,hypermarket,minimarket,supermarket
category,,,
Fabric Care,"19,328.14","17,757.14","17,847.56"
Rice,"71,205.46","67,135.57","74,921.18"
Sugar/Flavored Syrup,"13,539.92","12,352.14","13,071.11"


### Apa kegunaan parameter margins=True pada method `pd.crosstab`?

margins=True pada crosstab digunakan untuk menjumlahkan hasil cross tabulasi, sehingga ada kolom tambahan dengan nama kolom “All” yang berisi total nilai

In [64]:
pd.crosstab(
    index=household['category'],
    columns=household['format'],
    values=household['unit_price'],
    aggfunc='mean',
    margins=True
)

format,hypermarket,minimarket,supermarket,All
category,,,,
Fabric Care,"19,328.14","17,757.14","17,847.56","17,893.79"
Rice,"71,205.46","67,135.57","74,921.18","70,013.15"
Sugar/Flavored Syrup,"13,539.92","12,352.14","13,071.11","12,645.07"
All,"27,079.47","23,460.18","27,457.15","24,830.78"


### Apakah kita dapat melakukan `sort_values()` pada kolom `All` saat parameter `margins=True`?

Bisa. Dengan catatan yang dapat di *sorting* adalah nilai `All` pada kolomnya, bukan nilai `All` pada barisnya. 

## Missing Values

### Bagaimana menampilkan data yang memiliki missing values saja?

In [65]:
household = pd.read_csv("data_input/household.csv", index_col=1, parse_dates=['purchase_time'])
household.drop(['receipt_id', 'yearmonth', 'sub_category'], axis=1, inplace=True)
household['weekday'] = household['purchase_time'].dt.day_name()
import math
x=[i for i in range(32000000, 32000005)]
x.insert(2,32030785)
household2 = household.head(6).copy()
household2 = household2.reindex(x)
household2 = pd.concat([household2, household.head(14)])
household2.loc[31885876, "weekday"] = math.nan
# household2.iloc[2:8,]

Di bawah ini adalah contoh sebuah DataFrame yang memiliki beberapa *missing values* didalamnya. 

In [66]:
household2

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32000000,NaT,NaN,NaN,nan,nan,nan,NaN
32000001,NaT,NaN,NaN,nan,nan,nan,NaN
32030785,2018-07-17 18:05:00,Rice,minimarket,"63,500.00",0.00,1.00,Tuesday
32000002,NaT,NaN,NaN,nan,nan,nan,NaN
32000003,NaT,NaN,NaN,nan,nan,nan,NaN
32000004,NaT,NaN,NaN,nan,nan,nan,NaN
32369294,2018-07-22 21:19:00,Rice,supermarket,"128,000.00",0.00,1.00,Sunday
31885876,2018-07-15 16:17:00,Rice,minimarket,"102,750.00",0.00,1.00,NaN
31930241,2018-07-15 12:12:00,Rice,supermarket,"64,000.00",0.00,3.00,Sunday


Berikut adalah kode program untuk menampilkan data yang memiliki *missing values* saja.
`isna()` digunakan untuk melakukan pengecekan *missing values*, `.any(axis = 1)` mengecek apakah terdapat *missing values* pada kolom (yang diwakili dengan `axis=1`).

In [67]:
household2[household2.isna().any(axis = 1)]

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32000000,NaT,NaN,NaN,nan,nan,nan,NaN
32000001,NaT,NaN,NaN,nan,nan,nan,NaN
32000002,NaT,NaN,NaN,nan,nan,nan,NaN
32000003,NaT,NaN,NaN,nan,nan,nan,NaN
32000004,NaT,NaN,NaN,nan,nan,nan,NaN
31885876,2018-07-15 16:17:00,Rice,minimarket,"102,750.00",0.00,1.00,NaN


### Bagaimana cara menangani *missing values* pada data?

Terdapat berbagai cara menangani data yang mengandung *missing value*. Cara yang paling sering digunakan untuk menangani data yang mengandung *missing value*, yaitu *deletion*, *full analysis*, dan *imputation*.

**a. Deletion**

Deletion adalah membuang variabel/kolom pada data yang memiliki jumlah *missing value* (NA) melebihi 50% dari jumlah observasi. Hal ini menganggap bahwa variabel tersebut tidak banyak memberikan informasi pada data (variansinya mendekati 0). Ketika membuang variabel perlu memperhatikan business case dari data tersebut, apakah variabel yang dibuang akan menghilangkan informasi yang cukup signfikan atau tidak? Apakah ketika variabel tersebut dibuang ada informasi yang berkurang dari data tersebut atau tidak?

**b. Full analysis**
Full analysis adalah membuang observasi/baris yang mengandung *missing value*. Cara ini dilakukan jika jumlah observasi yang mengandung *missing value* NA tidak melebihi 5% dari total observasi data.

**c. Imputation**

Jika jumlah missing value pada data cukup banyak (melebihi 5% jumlah observasi), kita dapat melakukan imputation yaitu mengisi missing value tersebut dengan suatu nilai tertentu. Biasanya imputation dilakukan bedasarkan business knowledge dari variabel tersebut, misalkan variabel jumlah pengunjung per jam pada sebuah restoran. Seharusnya terdapat jumlah visitor yang bernilai 0 pada jam-jam tertentu karena pasti terdapat kemungkinan pada jam-jam tertentu tidak terdapat pengunjung. Tetapi, jika secara business knowledge seharusnya observasi tersebut memiliki suatu nilai, kita bisa melakukan imputation dengan menggunakan pusat datanya seperti mean/median untuk variabel numerik dan modus untuk variabel kategorik.